In [1]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *
import time

# Initialize YOLO model
model = YOLO('yolov8s.pt')

# Define the mouse event function for printing mouse position
def print_mouse_position(event, x, y, flags, param):
    """
    Print the mouse position when the mouse is moved.

    Parameters:
        event (int): The type of mouse event.
        x (int): The x-coordinate of the mouse position.
        y (int): The y-coordinate of the mouse position.
        flags (int): Additional flags from OpenCV.
        param (Any): Additional parameters passed to the callback function.
    """
    if event == cv2.EVENT_MOUSEMOVE:
        position = [x, y]
        print(position)

# Set up the OpenCV window and mouse callback
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', print_mouse_position)

# Open video capture
cap = cv2.VideoCapture("C:/Users/ASUS/Downloads/project/veh2.mp4")

# Load class names from coco.txt
my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")

# Initialize tracker
tracker = Tracker()

# Define constants
cy1 = 322
cy2 = 368
offset = 6

# Initialize dictionaries for tracking vehicles going up and down
vehicles_down = {}
vehicles_counted_down = []
vehicles_up = {}
vehicles_counted_up = []

# Main loop for processing frames
while True:    
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (1020, 500))

    # Perform object detection using YOLO
    results = model.predict(frame)
    a = results[0].boxes.data.cpu().numpy()  # Move tensor to CPU and convert to NumPy array
    px = pd.DataFrame(a).astype("float")
    list = []
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            list.append([x1, y1, x2, y2])  
    
    # Update tracker with detected objects
    bbox_id = tracker.update(list)
    
    # Process each tracked object
    for bbox in bbox_id:
        x3, y3, x4, y4, vehicle_id = bbox
        centroid_x = int((x3 + x4) / 2)
        centroid_y = int((y3 + y4) / 2)
        
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2)

        # Track vehicles going down
        if cy1 - offset < centroid_y < cy1 + offset:
            vehicles_down[vehicle_id] = time.time()
        if vehicle_id in vehicles_down:
            if cy2 - offset < centroid_y < cy2 + offset:
                elapsed_time = time.time() - vehicles_down[vehicle_id]
                if vehicle_id not in vehicles_counted_down:
                    vehicles_counted_down.append(vehicle_id)
                    distance = 10  # meters
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6
                    cv2.circle(frame, (centroid_x, centroid_y), 4, (0, 0, 255), -1)
                    cv2.putText(frame, str(vehicle_id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(a_speed_kh)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
        
        # Track vehicles going up
        if cy2 - offset < centroid_y < cy2 + offset:
            vehicles_up[vehicle_id] = time.time()
        if vehicle_id in vehicles_up:
            if cy1 - offset < centroid_y < cy1 + offset:
                elapsed1_time = time.time() - vehicles_up[vehicle_id]
                if vehicle_id not in vehicles_counted_up:
                    vehicles_counted_up.append(vehicle_id)
                    distance1 = 10  # meters
                    a_speed_ms1 = distance1 / elapsed1_time
                    a_speed_kh1 = a_speed_ms1 * 3.6
                    cv2.circle(frame, (centroid_x, centroid_y), 4, (0, 0, 255), -1)
                    cv2.putText(frame, str(vehicle_id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(a_speed_kh1)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    # Draw lines and labels for counting vehicles
    cv2.line(frame, (274, cy1), (814, cy1), (255, 255, 255), 1)
    cv2.putText(frame, ('L1'), (277, 320), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
    cv2.line(frame, (177, cy2), (927, cy2), (255, 255, 255), 1)
    cv2.putText(frame, ('L2'), (182, 367), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
    
    # Display the count of vehicles going up and down
    c = len(vehicles_counted_down)
    u = len(vehicles_counted_up)
    cv2.putText(frame, ('goingdown:-') + str(c), (60, 90), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
    cv2.putText(frame, ('goingup:-') + str(u), (60, 130), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    # Display the frame
    cv2.imshow("RGB", frame)
    
    # Break the loop if 'Esc' key is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()



0: 320x640 1 person, 3 cars, 1 fire hydrant, 62.5ms
Speed: 0.0ms preprocess, 62.5ms inference, 88.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 10.7ms
Speed: 7.5ms preprocess, 10.7ms inference, 15.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 15.3ms
Speed: 0.0ms preprocess, 15.3ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 12.8ms
Speed: 2.0ms preprocess, 12.8ms inference, 1.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 13.4ms
Speed: 2.0ms preprocess, 13.4ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 13.0ms
Speed: 2.1ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 12.6ms
Speed: 2.0ms preprocess, 12.6ms inference, 2.